<a href="https://colab.research.google.com/github/SJinLee/cryptology/blob/main/05.AES_GF256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.cs.uaf.edu/2015/spring/cs463/lecture/03_23_AES.html

In [17]:
def to_binary_list(m,ndigits=8):
    b = []
    for i in range(ndigits):
        b = [m%2] + b
        m = m//2
    return b
def to_num(b):
    m = 0
    for n in b:
        m = m*2+n
    return m

In [22]:
def add_gf256(a,b):
    return a^b
def mult_gf256(a,b):
    irr = 0x11b
    high = 0x100
    c = 0
    for bit in range(7,-1,-1):
        if b&(1<<bit):
            c=c^(a<<bit)

    for bit in range(7,-1,-1):
        if c & (high<<bit):
            c=c^(irr<<bit)
    return c

In [23]:
a = 0x57
b = 0x83
c = add_gf256(a,b)
hex(c)

'0xd4'

In [24]:
a = 0x57
b = 0x83
c = mult_gf256(a,b)
hex(c)

'0xc1'

In [21]:
to_binary_list(c)

[1, 1, 0, 0, 0, 0, 0, 1]